In [140]:
!pip3 install numpy nltk fastprogress

In [5]:
# python3.5 buildtagger.py <train_file_absolute_path> <model_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def train_model(train_file, model_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    train_file = sys.argv[1]
    model_file = sys.argv[2]
    start_time = datetime.datetime.now()
    train_model(train_file, model_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


Finished...
Time: 0:00:00.000121


In [5]:
# python3.5 runtagger.py <test_file_absolute_path> <model_file_absolute_path> <output_file_absolute_path>

import os
import math
import sys
import datetime
import numpy as np


def tag_sentence(test_file, model_file, out_file):
    # write your code here. You can add functions as well.
    print('Finished...')

if __name__ == "__main__":
    # make no changes here
    test_file = sys.argv[1]
    model_file = sys.argv[2]
    out_file = sys.argv[3]
    start_time = datetime.datetime.now()
    tag_sentence(test_file, model_file, out_file)
    end_time = datetime.datetime.now()
    print('Time:', end_time - start_time)


IndexError: list index out of range

## Creating the model statistics

Need to handle special cases like these:
 - fractions like: 1/2/CD
 - multiple listings: pianist/bassoonist/composer/NN
 
 As far as I understand all of the words belong to the same POS. 
 A function that adds both words should be implemented, but an initial version function should simply take the last two elements and uses them as word + tag.
 
 
 Also, likely the score could be improved by adding add-one-smoothing

In [209]:
import itertools
flatten = lambda x: list(itertools.chain.from_iterable(x))

class PosModelStats():
    def __init__(self, emissions, transitions, words, tags):
        self.emissions = emissions
        self.transitions = transitions
        self.words = words
        self.tags = tags
        

def extract_frequencies(filename):
    emission_dict = {}
    transition_dict = {}
    unique_words = set()
    unique_tags = set()

    with open(filename) as training_file:
        word_tag_pairs = [sentence.split(" ") for sentence in training_file.read().split("\n")]
        word_tag_pairs = flatten(word_tag_pairs)
        word_count = len(word_tag_pairs)
        previous_tag = word_tag_pairs[0].split("/")[-1]

        for word_tag in word_tag_pairs:
            if len(word_tag.split("/")) < 2:
                continue

            word, tag = word_tag.split("/")[-2:]

            unique_words.add(word)
            unique_tags.add(tag)

            if word_tag not in emission_dict.keys():
                emission_dict[word_tag] = 1
            else:
                emission_dict[word_tag] += 1

            if previous_tag + " " + tag not in transition_dict.keys():
                transition_dict[previous_tag + " " + tag] = 1
            else:
                transition_dict[previous_tag + " " + tag] += 1

            previous_tag = tag
    
    return PosModelStats(emission_dict, transition_dict, unique_words, unique_tags)


def add_one_smoothing():
    pass

In [229]:
class Dictionary():
    def __init__(self):
        self.word_dict = {}
        self.tag_dict = {}
        
        self.word_count = 0
        self.tag_count = 0
    
    def add_word(self, word):
        if word not in self.word_dict.values():
            self.word_dict[self.word_count] = word
            self.word_count += 1
            return self.word_count - 1
        return self.get_word_id(word)
    
    def add_tag(self, tag):
        if tag not in self.tag_dict.values():
            self.tag_dict[self.tag_count] = tag
            self.tag_count += 1
            return self.tag_count - 1
        return self.get_tag_id(tag)
            
    def get_word_id(self, word):
        for key, known_word in self.word_dict.items():
            if word == known_word:
                return key
    
    def get_tag_id(self, tag):
        for key, known_tag in self.tag_dict.items():
            if tag == known_tag:
                return key
    
    def get_word_tag_tuple(self, word_tag):
        words_tag = word_tag.split("/")
        assert len(words_tag) > 1, "Bad format for word/tag: {}".format(word_tag)
        
        word_ids = [self.get_word_id(word) for word in words_tag[:-1]]
        tag_id = self.get_tag_id(words_tag[-1])
        word_ids.append(tag_id)
        return tuple(word_ids)
    
    
class PosStats():
    def __init__(self):
        self.emissions = {}
        self.transitions = {}
        self.dictionary = Dictionary()
        
    def add_word(self, word):
        return self.dictionary.add_word(word)
        
    def add_tag(self, tag):
        return self.dictionary.add_tag(tag)
        
    def add_emission(self, word_id, tag_id):
        if (word_id, tag_id) not in self.emissions.keys():
            self.emissions[(word_id, tag_id)] = 1
        else:
            self.emissions[(word_id, tag_id)] += 1
        
    def add_transition(self, previous_tag_id, tag_id):
        if (previous_tag_id, tag_id) not in self.transitions.keys():
            self.transitions[(previous_tag_id, tag_id)] = 1
        else:
            self.transitions[(previous_tag_id, tag_id)] += 1

            
def extract_frequencies_dict(filename):    
    pos_stats = PosStats()

    with open(filename) as training_file:
        word_tag_pairs = flatten([sentence.split(" ") for sentence in training_file.read().split("\n")])        
        previous_tag_id = pos_stats.add_tag(word_tag_pairs[0].split("/")[-1])

        for word_tag in progress_bar(word_tag_pairs):
            if len(word_tag.split("/")) < 2:
                continue

            word, tag = word_tag.split("/")[-2:]
            
            word_id = pos_stats.add_word(word)
            tag_id = pos_stats.add_tag(tag)
            
            pos_stats.add_emission(word_id, tag_id)
            pos_stats.add_transition(previous_tag_id, tag_id)

            previous_tag_id = tag_id
    
    return pos_stats
            

In [240]:
stats = extract_frequencies_dict('sents.train')

In [243]:
stats.transitions

{(0, 0): 2017,
 (0, 1): 32370,
 (1, 2): 9047,
 (2, 3): 1752,
 (3, 4): 7378,
 (4, 0): 32889,
 (0, 5): 547,
 (5, 1): 1014,
 (1, 4): 38873,
 (4, 6): 686,
 (6, 0): 899,
 (0, 2): 14756,
 (2, 7): 5094,
 (7, 2): 905,
 (2, 2): 34528,
 (2, 8): 313,
 (8, 5): 42,
 (5, 9): 75,
 (9, 10): 760,
 (10, 11): 5077,
 (11, 1): 1473,
 (2, 12): 12688,
 (12, 6): 2796,
 (6, 4): 299,
 (4, 13): 5369,
 (13, 10): 1714,
 (10, 14): 98,
 (14, 12): 225,
 (12, 1): 6489,
 (12, 9): 993,
 (9, 0): 7331,
 (12, 15): 2555,
 (15, 16): 2519,
 (16, 9): 2431,
 (9, 17): 2142,
 (17, 2): 1009,
 (2, 18): 4987,
 (18, 2): 7360,
 (2, 19): 3532,
 (19, 2): 1301,
 (2, 20): 1530,
 (20, 2): 750,
 (2, 15): 6013,
 (15, 21): 1800,
 (21, 19): 3875,
 (19, 22): 261,
 (22, 2): 408,
 (2, 10): 2220,
 (10, 17): 2384,
 (17, 23): 12851,
 (23, 24): 2239,
 (24, 0): 3405,
 (0, 3): 5847,
 (3, 10): 5737,
 (10, 0): 14060,
 (3, 18): 2634,
 (18, 1): 8093,
 (4, 4): 16252,
 (4, 24): 1179,
 (24, 4): 27550,
 (4, 15): 6310,
 (15, 3): 1979,
 (12, 4): 2404,
 (13, 24):

In [239]:
stats.dictionary.tag_dict.items()

dict_items([(0, 'IN'), (1, 'DT'), (2, 'NNP'), (3, 'CD'), (4, 'NN'), (5, '``'), (6, "''"), (7, 'POS'), (8, '-LRB-'), (9, 'VBN'), (10, 'NNS'), (11, 'VBP'), (12, ','), (13, 'CC'), (14, '-RRB-'), (15, 'VBD'), (16, 'RB'), (17, 'TO'), (18, '.'), (19, 'VBZ'), (20, 'NNPS'), (21, 'PRP'), (22, 'PRP$'), (23, 'VB'), (24, 'JJ'), (25, 'MD'), (26, 'VBG'), (27, 'RBR'), (28, ':'), (29, 'WP'), (30, 'WDT'), (31, 'JJR'), (32, 'PDT'), (33, 'RBS'), (34, 'WRB'), (35, 'JJS'), (36, '$'), (37, 'RP'), (38, 'FW'), (39, 'EX'), (40, 'SYM'), (41, '#'), (42, 'LS'), (43, 'UH'), (44, 'WP$')])

## The combo one

In [252]:
class Dictionary():
    def __init__(self):
        self.word_dict = {}
        self.tag_dict = {}
        
        self.word_count = 0
        self.tag_count = 0
    
    def add_word(self, word):
        if word not in self.word_dict.values():
            self.word_dict[self.word_count] = word
            self.word_count += 1
            return self.word_count - 1
        return self.get_word_id(word)
    
    def add_tag(self, tag):
        if tag not in self.tag_dict.values():
            self.tag_dict[self.tag_count] = tag
            self.tag_count += 1
            return self.tag_count - 1
        return self.get_tag_id(tag)
            
    def get_word_id(self, word):
        for key, known_word in self.word_dict.items():
            if word == known_word:
                return key
    
    def get_tag_id(self, tag):
        for key, known_tag in self.tag_dict.items():
            if tag == known_tag:
                return key
    
    def get_word_tag_tuple(self, word_tag):
        words_tag = word_tag.split("/")
        assert len(words_tag) > 1, "Bad format for word/tag: {}".format(word_tag)
        
        word_ids = [self.get_word_id(word) for word in words_tag[:-1]]
        tag_id = self.get_tag_id(words_tag[-1])
        word_ids.append(tag_id)
        return tuple(word_ids)
    
    
class PosStatsText():
    def __init__(self):
        self.emissions = {}
        self.transitions = {}
        self.dictionary = Dictionary()
        
    def add_tag(self, tag):
        return self.dictionary.add_tag(tag)
        
    def add_emission(self, word_id, tag_id):
        if (word_id, tag_id) not in self.emissions.keys():
            self.emissions[(word_id, tag_id)] = 1
        else:
            self.emissions[(word_id, tag_id)] += 1
        
    def add_transition(self, previous_tag_id, tag_id):
        if (previous_tag_id, tag_id) not in self.transitions.keys():
            self.transitions[(previous_tag_id, tag_id)] = 1
        else:
            self.transitions[(previous_tag_id, tag_id)] += 1

            
def extract_frequencies_tuple_text(filename):    
    pos_stats = PosStatsText()

    with open(filename) as training_file:
        word_tag_pairs = flatten([sentence.split(" ") for sentence in training_file.read().split("\n")])        
        previous_tag_id = pos_stats.add_tag(word_tag_pairs[0].split("/")[-1])

        for word_tag in progress_bar(word_tag_pairs):
            if len(word_tag.split("/")) < 2:
                continue

            word, tag = word_tag.split("/")[-2:]
            tag_id = pos_stats.add_tag(tag)
            
            pos_stats.add_emission(word, tag_id)
            pos_stats.add_transition(previous_tag_id, tag_id)

            previous_tag_id = tag_id
    
    return pos_stats
            

In [253]:
text_stats = extract_frequencies_tuple_text('sents.train')

In [255]:
text_stats.dictionary.tag_dict

{0: 'IN',
 1: 'DT',
 2: 'NNP',
 3: 'CD',
 4: 'NN',
 5: '``',
 6: "''",
 7: 'POS',
 8: '-LRB-',
 9: 'VBN',
 10: 'NNS',
 11: 'VBP',
 12: ',',
 13: 'CC',
 14: '-RRB-',
 15: 'VBD',
 16: 'RB',
 17: 'TO',
 18: '.',
 19: 'VBZ',
 20: 'NNPS',
 21: 'PRP',
 22: 'PRP$',
 23: 'VB',
 24: 'JJ',
 25: 'MD',
 26: 'VBG',
 27: 'RBR',
 28: ':',
 29: 'WP',
 30: 'WDT',
 31: 'JJR',
 32: 'PDT',
 33: 'RBS',
 34: 'WRB',
 35: 'JJS',
 36: '$',
 37: 'RP',
 38: 'FW',
 39: 'EX',
 40: 'SYM',
 41: '#',
 42: 'LS',
 43: 'UH',
 44: 'WP$'}

In [194]:
%%time

pos_model_stats = extract_frequencies('sents.train')


CPU times: user 2.9 s, sys: 818 ms, total: 3.72 s
Wall time: 4.09 s


In [71]:
import _pickle as pickle

def write_output(model_stats, filename="model-file"):
    with open(filename, "wb") as model_file:
        model_file.write(pickle.dumps(model_stats))

In [72]:
%%time

write_output(pos_model_stats)

CPU times: user 50.7 ms, sys: 8.89 ms, total: 59.6 ms
Wall time: 62.8 ms


In [110]:
with open("model-file", "rb") as model_read:
    loaded = pickle.load(model_read)

# Simple implementation of POS tagger

In [143]:
def read_and_tag(input_filename, pos_tagger):
    tagged_sent = []

    with open(input_filename, "r") as input_file:
        sentences = input_file.read().split("\n")

        for sentence in progress_bar(sentences):
            tagged_sent.append(pos_tagger(sentence))
    
    return tagged_sent

## Problems

How do I deal with words that have yet to be seen in the training data?
- Possibly remove capitalization before collecting stats?

Very slow performance
- Create a dictionary of words and use the word embeddings instead of a dictionary

Need to calculate the begining of the sentence transition as well

Possibly ignore dictionary creation, but use tuples instead of the initial implementation of the training algorithm

In [154]:
from functools import partial
from fastprogress import progress_bar

common_tag = "NN"

def word_emissions(word, keys):
    return [key for key in keys if key.startswith(word)]

def simple_pos_tagger(sentence, stats):
    tagged_sentence = []
    keys = stats.emissions.keys()
    
    for word in sentence.split(" "):
        emission_tags = word_emissions(word, keys)
        emission_probs = [stats.emissions[word_tag] for word_tag in emission_tags]

        if len(emission_tags) > 0:
            tagged_sentence.append(emission_tags[np.argmax(emission_probs)])
        else:
#             print("Word hasn't appeared in the training data: {}".format(word))
            tagged_sentence.append("{}/{}".format(word, common_tag))
        
    return " ".join(tagged_sentence)

simple_stats_tagger = partial(simple_pos_tagger, stats=loaded)

In [156]:
%%time

tagged = read_and_tag("sents.test", pos_tagger=simple_stats_tagger)

CPU times: user 9min 7s, sys: 3.54 s, total: 9min 11s
Wall time: 9min 32s


In [160]:
with open("simple_test_output.txt", "w") as out_file:
    out_file.write("\n".join(tagged[:-1]))

In [267]:
sentence = "I am a test sentence ."
pos_tags = 45
words = sentence.split(" ")

word_dict = {
    0: "fake",
    1: "news",
    2: "great",
}

pos_dict = {
    0: "NNP",
    1: "IT",
    2: "TH",
}

transition_probs = {
    (0, 0): 0.10,
    (0, 1): 0.10,
    (0, 2): 0.30,
    (1, 0): 0.20,
    (2, 1): 0.30,
}

emission_probs = {
    (0, 0): 0.10,
    (0, 1): 0.10,
    (0, 2): 0.30,
    (1, 0): 0.20,
    (2, 1): 0.30,
}

In [395]:
class PosStatsModel():
    def __init__(self, pos_stats=None):
        self.emissions = {}
        self.transitions = {}
        self.dictionary = Dictionary()
        self.MIN_PROB_VALUE = 10e-5
        
        if pos_stats is not None:
            self.emissions = pos_stats.emissions
            self.transitions = pos_stats.transitions
            self.dictionary = pos_stats.dictionary
        
    def add_tag(self, tag):
        return self.dictionary.add_tag(tag)
    
    def get_all_tag_ids(self):
        return self.dictionary.tag_dict.keys()
        
    def add_emission(self, word_id, tag_id):
        if (word_id, tag_id) not in self.emissions.keys():
            self.emissions[(word_id, tag_id)] = 1
        else:
            self.emissions[(word_id, tag_id)] += 1
        
    def add_transition(self, previous_tag_id, tag_id):
        if (previous_tag_id, tag_id) not in self.transitions.keys():
            self.transitions[(previous_tag_id, tag_id)] = 1
        else:
            self.transitions[(previous_tag_id, tag_id)] += 1
    
    def get_emission_prob(self, word, tag):
        try:
            return self.emissions[(word, tag)]
        except KeyError:
            return self.MIN_PROB_VALUE
        
    def get_transition_prob(self, prev_tag, tag):
        try:
            return self.transitions[(prev_tag, tag)]
        except KeyError:
            return self.MIN_PROB_VALUE
    
    
def get_transition_probs(previous_probs, pos_stats, tags):
    max_ids = []
    probs = []
    
    for tag_id in tags:
        transitions = [pos_stats.get_transition_prob(prev_tag_id, tag_id) for prev_tag_id in tags]
        transitions_with_previous = previous_probs * np.array(transitions)
        max_id = np.argmax(transitions_with_previous)
        max_ids.append(max_id)
        probs.append(transitions_with_previous[max_id])
        
    return probs, max_ids

def combine_words_tags(words, tag_list, pos_stats):
    for i in range(len(words)):
        words[i] = words[i] + "/" + pos_stats.dictionary.tag_dict[tag_list[i]]
    
    return " ".join(words)
    
# Check speed/performance for int/float
def get_sentence_tags(sentence, pos_stats):
    pos_tags = pos_stats.get_all_tag_ids()
    words = sentence.split(" ")

    viterbi = np.zeros((len(pos_tags), len(words)), float)
    backpointers = np.zeros((len(pos_tags), len(words)), int)

    # fill in initial probs
    # for now skipping the beginning of the sentence probability

    viterbi[:, 0] = [pos_stats.get_emission_prob(words[0], tag_id) for tag_id in pos_tags]
    backpointers[:, 0] = -1

    # fill in the full matrix
    for i, word in enumerate(words[1:]):
        emission_probs = [pos_stats.get_emission_prob(word, tag_id) for tag_id in pos_tags]
        transition_probs, new_backpointers = get_transition_probs(viterbi[:, i], pos_stats, pos_tags)

        viterbi[:, i + 1] = np.array(emission_probs) * np.array(transition_probs)
        backpointers[:, i + 1] = new_backpointers

    # Also need to calculate the ending probabilities
    tag_list = np.zeros(len(words) + 1, int)
    
    final_tag = np.argmax(viterbi[:, -1]) # this should be in accordance to the end of the sentence
    tag_list[-1] = final_tag
    previous_back = final_tag

    for word_pos in reversed(range(1, len(words))):
        tag_list[word_pos] = backpointers[previous_back, word_pos]
        previous_back = tag_list[word_pos]
        
    tag_list = tag_list[1:]
        
    return combine_words_tags(words, tag_list, pos_stats)

In [396]:
pos_stats = PosStatsModel(text_stats)
sentence = "I am a test sentence ."

In [397]:
%%timeit

get_sentence_tags(sentence, pos_stats)

11.6 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Implementing the new prediction model

In [400]:
hmm_viterbi_tagger = partial(get_sentence_tags, pos_stats=pos_stats)

In [401]:
%%time

tagged = read_and_tag("sents.test", pos_tagger=hmm_viterbi_tagger)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:82: RuntimeWarning: overflow encountered in multiply
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:50: RuntimeWarning: overflow encountered in multiply


CPU times: user 1min 46s, sys: 3.33 s, total: 1min 49s
Wall time: 1min 54s


In [404]:
with open("viterbi_test_output.txt", "w") as out_file:
    out_file.write("\n".join(tagged[:-1]))